# 创建情感分析网络应用
## 使用 PyTorch 和 SageMaker

_机器学习工程师纳米学位课程 | 部署_

---

我们已经基本了解了 SageMaker 的运行原理，下面将使用 SageMaker 从头到尾地完成一个项目。我们的目标是创建一个简单网页，用户可以在网页里输入影评。网页然后将影评发送给部署的模型，模型将预测影评的情感。

## 说明

我们已经提供了一些模板代码，但是你需要实现其他功能，才能成功地完成此 notebook。除了要求的部分之外，不需要修改所包含的代码。标题以“**TODO**”开头的部分表示你需要完成或实现其中的某些部分。我们将在每个部分提供说明，并在代码块中用 `# TODO: ...` 注释标记出具体的实现要求。请务必仔细阅读说明。

除了实现代码之外，你还需要回答一些问题，这些问题与任务和你的实现代码有关。每个部分需要回答的问题都在标题中以“**问题：**”开头。请仔细阅读每个问题，并编辑下面以“**答案：**”开头的标记单元格，然后输入答案。

> 注意：可以通过 **Shift+Enter** 键盘快捷键执行代码和标记单元格。此外，通常还可通过点击单元格（标记单元格需要双击）编辑单元格，或者在选中后按下 **Enter** 键编辑单元格。

## 一般步骤

复习下在 notebook 实例中创建 SageMaker 项目的一般步骤。

1. 下载或检索数据。
2. 处理/准备数据。
3. 将处理的数据上传到 S3。
4. 训练所选的模型。
5. 测试训练的模型（通常使用批转换作业）。
6. 部署训练的模型。
7. 使用部署的模型。

对于此项目，你将按照一般步骤中列出的步骤操作，不过会稍加修改。

首先，不用在单独的步骤中测试模型。依然会测试模型，但是将部署模型，然后向部署的模型发送数据。这么做的原因之一是可以检查确保部署的模型能正常运行，然后再继续。

此外，你将再次部署和使用训练过的模型。第二次你将添加一些代码，自定义训练的模型的部署方式。此外，新部署的模型将用在情感分析网络应用中。

## 第 1 步：下载数据

与 SageMaker 中的 XGBoost notebook 一样，我们将使用 [IMDB 数据集](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2020-05-13 15:21:51--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  24.1MB/s    in 4.0s    

2020-05-13 15:21:55 (20.3 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## 第 2 步：准备和处理数据

和 XGBoost notebook 一样，我们将先处理数据。前几步与 XGBoost 示例一样。首先，读取每条影评并组合成一个输入结构。然后，将数据集拆分成训练集和测试集。

In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


从下载的数据集中读取原始训练和测试数据后，我们将组合正面和负面影评，并随机排列数据记录。

In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


汇总和准备好训练及测试集后，我们应该快速检查下并看一个模型训练数示例。通常建议检查一下，因为可以了解后续处理步骤对影评的影响，并且可以检查数据是否加载正确。

In [6]:
print(train_X[100])
print(train_y[100])

This is without a doubt the funniest of the Curly stooges shorts. I've seen it dozens of times and it always makes me laugh. Hilarious pantomime sequences. A perfect example of "musical comedy". Even people who don't like the knuckleheads remember this one fondly.
1


处理影评的第一步是删除所有的 HTML 标记。还需要标记化输入，使 *entertained* 和 *entertaining* 在情感分析时被视为相同的字词。

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

上面定义的 `review_to_words` 方法会使用 `BeautifulSoup` 删除所有的 HTML 标记，并使用 `nltk` 软件包标记化影评。为了检查一切是否都正常运行，尝试将 `review_to_words` 应用到训练集中的某个影评上。

In [8]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
review_to_words(train_X[100])

['without',
 'doubt',
 'funniest',
 'curli',
 'stoog',
 'short',
 'seen',
 'dozen',
 'time',
 'alway',
 'make',
 'laugh',
 'hilari',
 'pantomim',
 'sequenc',
 'perfect',
 'exampl',
 'music',
 'comedi',
 'even',
 'peopl',
 'like',
 'knucklehead',
 'rememb',
 'one',
 'fondli']

**问题：**我们在上面提到 `review_to_words` 方法会删除所有的 HTML 格式标记，并标记化影评中的字词，例如将 *entertained* 和 *entertaining* 转换为 *entertain*，并将它们视为同一个字词。此方法还对输入进行了什么处理？

**回答：**

将所有字母转换为小写字母，替换所有非字母数字为空字符串，将字符串分割为单词，并删除stopwords。


以下方法向训练集和测试集中的每条影评都应用了 `review_to_words` 方法。此外，它还会缓存结果。因为这个处理步骤的时间很长。如果你在当前会话中无法完成 notebook，可以稍后再继续，无需重新处理数据。

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## 转换数据

在 XGBoost notebook 中，我们将数据从字词表示法转换成了词袋特征表示法。对于我们将在此 notebook 中构建的模型，我们将构建一个非常相似的特征表示法。首先，我们将每个字词表示成整数。当然，影评中的某些字词出现频率太低，可能对于情感分析来说包含的信息很少。为了解决这一问题，我们将固定词汇表的大小，仅包含最常出现的字词。然后将所有不常出现的字词放入一个类别，标为 `1`。

因为我们将使用循环神经网络，所以每条影评的长度最好相同。我们将固定影评的大小，然后用类别“无字词”（标为 `0`）填充更短的影评，并截断更长的影评。

### (TODO) 创建字词字典

首先，我们需要将影评中的字词映射为整数。我们将词汇表（包括“无字词”和“不常见”类别）的大小固定为 `5000`，但是你也可以改成其他大小，看看对模型有何影响。

> **TODO：**请完成以下 `build_dict()` 方法的实现代码。注意，虽然 vocab_size 设为 `5000`，但是我们只需对前 `4998` 个字词进行映射。因为我们将用特殊标签 `0` 表示“无字词”和`1`表示“不常见字词”。

In [11]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for entry in data:
        for word in entry:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
                
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [12]:
word_dict = build_dict(train_X)

**问题：**训练集中的前五个字词是哪些字词（标记化）？这些字词在训练集中经常出现合理吗？

**回答：**

训练集中的前五个字词是'movi', 'film', 'one', 'like', 'time'。合理，这些词都是影评的常用词汇。

In [13]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
list(word_dict.keys())[:5]

['movi', 'film', 'one', 'like', 'time']

### 保存 `word_dict`

稍后当我们构建一个处理提交的影评的端点时，我们需要使用创建的 word_dict`。所以，我们将其保存到文件中以供后面使用。

In [14]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [15]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### 转换影评

我们创建了字词字典，它可以将影评中的字词转换成整数，下面使用它将影评转换成整数序列表示法，并通过填充或截断的方式变成固定长度，即 `500`。

In [16]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [17]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

为了快速检查一切是否按预期运行，看看在处理之后，训练集中的某个影评看起来如何。看起来合理吗？训练集中的影评长度是多少？

In [18]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.

train_X[100]

array([  94,   21,  310,  224,    4,    2,    1,   55,   87,  113, 1693,
        371,  612,   26,  148,   22,   86,    2,   95,    4,  174,   13,
         56,  224,    6,  966,    1,    1,    1,  371,  613, 3240, 2342,
          9,  110, 2287,    2, 1178, 2110,  924, 2503,  245, 2197,  114,
          1,   55,  161,  174,    1,    2,   56,  255,  100,  310,   76,
       1613,    2,  224,    6,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

**问题：**在上面的单元格中，我们使用 `preprocess_data` 和 `convert_and_pad_data` 方法同时处理了训练集和测试集。这样做有问题吗？为何？

**回答：** 


有问题，因为训练集和测试集需要以相同的方式处理，以确保我们度量模型的性能。

## 第 3 步：将数据上传到 S3

与 XGBoost notebook 一样，我们需要将训练数据集上传到 S3，使训练代码能够访问数据。暂时先保存到本地，稍后再上传到 S3。

### 将处理过的训练集保存到本地

一定要知道所保存的数据的格式，因为在编写训练代码时需要知道。数据集中的每行格式为 `label`, `length`, `review[500]`，其中 `review[500]` 是一个长度为 `500` 的整数序列，表示影评中的字词。

In [19]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### 上传训练数据


接着，我们需要将训练数据上传到 SageMaker 默认 S3 存储桶里，以便在训练模型时能够访问训练数据。

In [20]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [21]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**注意：**上面的单元格会上传整个数据字典，包括 `word_dict.pkl` 文件。这很有用，因为稍后当我们创建接受任意影评的端点时，我们将需要该文件。暂时我们只需注意到它位于数据字典中（并且位于 S3 训练存储桶中），并且需要确保它保存到了模型目录下。

## 第 4 步：构建和训练 PyTorch 模型

在 XGBoost notebook 中，我们讨论了模型在 SageMaker 框架中的含义。模型由三个对象组成：

 - 模型工件，
 - 训练代码，以及
 - 推理代码
 
它们会相互交互。在 XGBoost 示例中，我们使用了由 Amazon 提供的训练和推理代码。我们依然将使用由 Amazon 提供的容器，因为该容器使我们能够添加自定义代码。

我们将开始在 PyTorch 中使用训练脚本实现我们自己的神经网络。对于此项目，我们已经在 `train` 文件夹中的 `model.py` 文件里提供了必要的模型对象。你可以通过运行以下单元格查看提供的实现代码。

In [22]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

对于所提供的实现，要注意的主要是，为了改善模型的性能，我们可能需要更改三个参数：嵌入维度、隐藏维度和词汇表大小。我们可能需要使这些参数在训练脚本中可配置，这样的话，当我们想要修改这些参数时，不需要修改脚本本身。稍后我们将介绍如何做到这一点。首先，我们将在 notebook 中编写一些训练代码，从而更轻松地诊断任何问题。

首先，我们将加载一小部分训练数据集作为样本。在 notebook 中尝试完整地训练模型可能要花费很长时间，因为我们无法使用 GPU，我们所使用的计算实例并不强大。但是，我们可以使用一小部分数据，看看训练脚本的行为如何。

In [23]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### (TODO) 编写训练方法

接着，我们需要编写训练代码。代码与之前训练 PyTorch 模型时编写的训练方法应该很相似。稍后再解决模型保存/加载和超参数加载等疑难问题。

In [24]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            optimizer.zero_grad()
            out = model(batch_X)
            loss = loss_fn(out, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

假设我们有了上述训练方法，为了测试它能否正常运行，我们将在 notebook 中编写一小段代码，该代码会对之前加载的小型样本训练集执行训练方法。在 notebook 中这么做的原因时，尽早发现可能会出现的问题并且更容易诊断。

In [25]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6945248246192932
Epoch: 2, BCELoss: 0.6850423812866211
Epoch: 3, BCELoss: 0.6777895212173461
Epoch: 4, BCELoss: 0.6705937266349793
Epoch: 5, BCELoss: 0.6625313997268677


为了使用 SageMaker 构建 PyTorch 模型，我们必须向 SageMaker 提供训练脚本。我们还可以包含一个目录，该目录将被复制到容器中，并从中运行训练代码。当训练容器被执行时，它将检查上传的目录（如果有的话）中是否有 `requirements.txt` 文件，并安装任何必要的 Python 库，然后运行训练脚本。

### (TODO) 训练模型

在 SageMaker 中构建了 PyTorch 模型后，必须指定入口点。当模型被训练时，SageMaker 将执行该 Python 文件。在 `train` 目录里有个文件叫做 `train.py`，我们提供了该文件，并且其中包含训练代码所需的大部分必要代码。唯一缺少的是 `train()` 方法的实现代码，你之前已经在此 notebook 中编写过这些代码。

**TODO**: 将在上面编写的 `train()` 方法复制到 `train/train.py` 文件的正确位置。

SageMaker 通过参数形式将超参数传递给训练脚本。这些参数然后可以被解析并用在训练脚本中。你可以查看提供的 `train/train.py` 文件，看看是如何完成这些步骤的。

In [26]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [27]:
estimator.fit({'training': input_data})

2020-05-13 15:22:46 Starting - Starting the training job...
2020-05-13 15:22:49 Starting - Launching requested ML instances......
2020-05-13 15:23:53 Starting - Preparing the instances for training.........
2020-05-13 15:25:29 Downloading - Downloading input data...
2020-05-13 15:25:53 Training - Downloading the training image...
2020-05-13 15:26:33 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-13 15:26:34,270 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-13 15:26:34,295 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-13 15:26:34,946 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-13 15:26:35,172 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-05-13 15:26:35,172 sagem

Model loaded with embedding_dim 32, hidden_dim 200, vocab_size 5000.
Epoch: 1, BCELoss: 0.6706523104589812
Epoch: 2, BCELoss: 0.6061840045208834
Epoch: 3, BCELoss: 0.5238828914506095
Epoch: 4, BCELoss: 0.45488954381066926
Epoch: 5, BCELoss: 0.3895870374173534
Epoch: 6, BCELoss: 0.3653721170766013
Epoch: 7, BCELoss: 0.32843537354955865
Epoch: 8, BCELoss: 0.3114958235195705
Epoch: 9, BCELoss: 0.29623928574883207
Epoch: 10, BCELoss: 0.28885673077739016
2020-05-13 15:29:58,083 sagemaker-containers INFO     Reporting training SUCCESS

2020-05-13 15:30:01 Uploading - Uploading generated training model
2020-05-13 15:30:35 Completed - Training job completed
Training seconds: 306
Billable seconds: 306


## 第 5 步：测试模型

正如在此 notebook 开头提到的，为了测试该模型，我们首先将部署模型，然后向部署的端点发送测试数据。这样可以检查部署的模型是否能正常运行。

## 第 6 步：部署模型以测试模型

训练模型后，我们想要测试下模型的效果。目前模型要求输入格式为 `review_length, review[500]`，其中 `review[500]` 是长度为 `500` 的整数序列，描述了影评中的字词，并使用 `word_dict` 编码。幸运的是，SageMaker 为需要这样的简单输入的模型提供了内置推理代码。

但是我们还需要提供一项内容，即加载保存的模型的函数。此函数必须叫做 `model_fn()`，并且唯一输入参数是模型工件所在的目录路径。此函数还必须出现在指定为入口点的 python 文件中。对我们来说，已经提供了模型加载函数，所以不需要进行更改。

**注意**：当内置推理代码运行时，它必须从 `train.py` 文件导入 `model_fn()` 方法。所以训练代码封装在了 main 关键字中（例如 `if __name__ == '__main__':` ）

因为我们不需要更改在训练中上传的代码，所以原样部署当前模型。

**注意：**部署模型时，我们将要求 SageMaker 启动一个计算实例，该实例将等待接收数据。所以，此实例将一直运行，直到你关闭它。这一点很重要，因为部署的端点按照运行时长计费。

换句话说，**如果你不再使用部署的端点，请关闭！**

**TODO:** 部署训练过的模型。

In [30]:
# TODO: Deploy the trained model
predictor = estimator.deploy(initial_instance_count = 1, instance_type= 'ml.m4.xlarge')

Using already existing model: sagemaker-pytorch-2020-05-13-15-22-46-341


---------------!

## 第 7 步：使用模型进行测试

部署模型后，我们可以读入测试数据并将其发送给部署的模型以获得一些结果。收集了所有结果后，我们可以判断模型的准确率如何。

In [31]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [32]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [33]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.84764

**问题：**这个模型与之前创建的 XGBoost 模型相比效果如何？为何这两个模型在此数据集上的效果不一样？你认为哪个模型在情感分析方面效果更好？

**回答：**

之前创建的XGBoost的准确率略高于0.86，这个模型的准确率只有0.84764，表现稍微差一些。

XGBoost在数据量较小的回归和分类问题执行良好。而神经网络深度模型需要大量的训练数据才能实现良好的性能。

在情感分析方面，我认为随着时间的推移及数据量的增加，神经网络的效果会更好。

### (TODO) 更多测试

我们已经有了训练过的模型，该模型已部署，并且我们可以向其发送处理过的影评，它将返回预测的情感。但是，最终我们希望能够向模型发送未处理过的影评，即发送字符串形式的影评。例如，假设我们想向模型发送以下影评。

In [35]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

问题是，如何将此影评发送给模型？

在此 notebook 的第一部分，我们对 IMDB 数据集执行了多个数据处理步骤。我们对提供的影评执行了以下两项处理：
 - 删除 HTML 标记并词干化输入
 - 使用 `word_dict` 将影评转换成整数序列
 
为了处理影评，我们需要重复这两个步骤。

**TODO**：使用第一部分的 `review_to_words` 和 `convert_and_pad` 方法将 `test_review` 变成适合发送给模型的 numpy 数组 `test_data`。模型要求输入格式为 `review_length, review[500]`。

In [36]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data

data_X ,data_len = convert_and_pad(word_dict, review_to_words(test_review))
data_pack = np.hstack((data_len, data_X))
data_pack = data_pack.reshape(1, -1)
test_data = torch.from_numpy(data_pack)

处理了影评后，我们可以将生成的数组发送给模型，以预测影评的情感。

In [37]:
predictor.predict(test_data)

array(0.95467514, dtype=float32)

因为模型的返回值接近 `1`，所以确定我们提交的影评是正面的。

### 删除端点

与 XGBoost notebook 一样，部署端点后，端点将继续运行，直到我们关闭端点。因为暂时不使用端点了，所以可以删除端点。

In [38]:
estimator.delete_endpoint()

## 第 6 步（再次）-针对网络应用部署模型

我们知道模型能正常运行，下面编写一些自定义推理代码，从而向模型发送未处理过的影评，并让模型预测影评的情感。

正如在上面看到的，默认情况下，我们创建的评估器在部署后，将使用我们在创建模型时提供的入口脚本和目录。但是，因为现在我们想接受字符串形式的输入，而模型要求输入是处理过的影评，所以需要编写自定义推理代码。

我们将编写的代码存储在 `serve` 目录中。此目录包含 `model.py` 文件，该文件将用于构建模型；以及 `utils.py` 文件，其中包含 `review_to_words` 和 `convert_and_pad` 预处理函数，我们在一开始的数据处理步骤中使用了这两个函数；还包含 `predict.py` 文件，其中包含自定义推理代码。注意，还有 `requirements.txt` 文件，它将告诉 SageMaker 自定义推理代码需要什么 Python 库。

在 SageMaker 中部署 PyTorch 模型时，你需要提供四个函数供 SageMaker 推理容器使用。
 - `model_fn`：此函数与我们在训练脚本中使用的函数一样，它将告诉 SageMaker 如何加载模型。
 - `input_fn`：此函数会接收发送给模型端点的原始序列化输入，它的职责是取消序列化输入，并且使推理代码能够使用输入。
 - `output_fn`：此函数会接受推理代码的输出，它的职责是序列化此输出并将其返回给模型端点的调用者。
 - `predict_fn`：推理脚本的核心，实际预测就发生在其中，你需要完成此函数。

对于我们要构建的简单网站，`input_fn` 和 `output_fn` 方法相对比较简单。我们只需接受字符串作为输入，并返回一个值作为输出。但是可以想象，在更复杂的应用中，输入或输出可能是图像数据或其他二元数据，需要一定的序列化操作。

### (TODO) 编写推理代码

在编写自定义推理代码之前，首先看看所提供的代码。

In [48]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

正如之前提到的，`model_fn` 方法与训练代码中提供的方法一样，`input_fn` 和 `output_fn` 方法很像，你的任务是完成 `predict_fn` 方法。记得将完成的文件保存为 `predict.py` 并放入 `serve` 目录中。

**TODO**：完成 `serve/predict.py` 文件中的 `predict_fn()` 方法。

### 部署模型

编写了自定义推理代码后，我们将创建并部署模型。首先，我们需要构建新的 PyTorchModel 对象，它将指向在训练过程中创建的模型工件，并指向我们要使用的推理代码。然后，我们可以调用部署方法来启动部署容器。

**注意**：部署的 PyTorch 模型的默认行为是，假设传递给预测器的任何输入是 `numpy` 数组。我们希望发送字符串，所以需要创建一个简单的 `RealTimePredictor` 类封装器，以包含简单的字符串。在更复杂的情形下，你可能需要提供序列化对象，例如如果想要发送图像数据的话。

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------

### 测试模型

部署了包含自定义推理代码的模型后，我们应该检查一切能否正常运行。我们将加载前 `250` 个正面和负面影评，并将它们发送给端点，然后收集结果。只发送部分数据的原因是，模型处理输入然后执行推理需要花费很长时间，所以不建议测试整个数据集。

In [51]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [52]:
ground, results = test_reviews()

Starting  pos  files


ValueError: invalid literal for int() with base 10: b'1.0'

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

还有一种测试方法是，发送之前查看过的 `test_review`。

In [ ]:
predictor.predict(test_review)

知道端点能按照预期运行后，我们将设置与端点交互的网页。如果你暂时没时间完成项目了，请跳到此 notebook 的末尾并关闭端点。之后可以再部署端点。

## 第 7 步（再次）：在网络应用中使用模型

> **TODO：**整个这一部分以及接下来要完成的任务主要用到 AWS 控制台。

到目前为止，我们一直通过构建一个使用端点的预测器对象访问模型端点，然后使用预测器对象进行推理。如果我们想创建一个访问模型的网络应用呢？目前的设置无法做到这一点，因为为了访问 SageMaker 端点，应用首先需要使用能够访问 SageMaker 端点的 IAM 角色向 AWS 验证身份。但是，还有一种更简单的方式。我们只需使用其他 AWS 服务。

<img src="Web App Diagram.svg">

上面的示意图解释了各种服务是如何协同工作的。最右边的是模型，我们在上面训练了模型，并且使用 SageMaker 部署了模型。最左侧的是网络应用，应用将收集用户的影评，将其发送给端点，并获得正面或负面情感预测。

中间的部分比较关键。我们将创建一个 Lambda 函数，你可以将其看做一个简单的 Python 函数，每次发生特定的事件时，该 Python 函数就会被执行。该函数有权向 SageMaker 端点发送数据及从中接收数据。

最后，我们执行 Lambda 函数所使用的是新的端点，我们将使用 API Gateway 创建该端点。此端点将是一个 URL，专门监听是否有数据发送给它。端点获得数据后，会将数据传递给 Lambda 函数，并返回 Lambda 函数返回的结果。它充当了网络应用与 Lambda 函数之间的通信接口。

### 设置 Lambda 函数

首先设置 Lambda 函数。每当公共 API 向其发送数据，该 Lambda 函数就会执行。当它被执行时，它将接收数据，对数据执行必要的处理步骤，将数据（影评）发送给我们创建的 SageMaker 端点，然后返回结果。

#### 第一部分：为 Lambda 函数创建 IAM 角色

因为我们希望 Lambda 函数调用 SageMaker 端点，所以需要确保它有权限这么做。我们将创建一个之后分配给 Lambda 函数的角色。

在 AWS 控制台中转到 **IAM** 页面并点击 **Roles**，然后点击 **Create role**。确保 **AWS service** 属于所选的受信实体类型，并选择 **Lambda** 作为将使用该角色的服务，然后点击 **Next: Permissions**。

在搜索框中输入 `sagemaker` 并选中 **AmazonSageMakerFullAccess** 策略旁边的复选框，然后点击 **Next: Review**。

最后命名此角色。设定一个后面能记住的名称，例如 `LambdaSageMakerRole`。然后点击 **Create role**。

#### 第二部分：创建 Lambda 函数

下面开始真正地创建 Lambda 函数。

首先在 AWS 控制台中转到 AWS Lambda 页面并点击 **Create a function**。在下个页面选择 **Author from scratch**。下面命名 Lambda 函数，使用后面能记住的名称，例如 `sentiment_analysis_func`。记得选中 **Python 3.6** 运行时，然后选择在上个部分创建的角色。接着点击 **Create Function**。

在下个页面，你将看到关于刚刚创建的 Lambda 函数的一些信息。向下滚动应该会看到一个编辑器，你可以在其中编写一些代码，当 Lambda 函数被触发时，这些代码将执行。在本示例中，我们将使用以下代码。

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

将上述代码复制粘贴到 Lambda 代码编辑器中后，将 `**ENDPOINT NAME HERE**` 部分替换成我们之前部署的端点的名称。你可以使用以下代码单元格获得端点的名称。

In [43]:
predictor.endpoint

'sagemaker-pytorch-2020-05-13-15-48-08-941'

将端点名称添加到 Lambda 函数中后，点击 **Save**。Lambda 函数现在已经在运行。接下来，我们需要让网络应用能够执行 Lambda 函数。

### 设置 API Gateway

Lambda 函数现在已设置好，下面使用 API Gateway 创建新的 API，它将触发我们刚刚创建的 Lambda 函数。

在 AWS 控制台中转到 **Amazon API Gateway**，然后点击 **Get started**。

在下个页面选中 **New API** 并命名新的 API，例如 `sentiment_analysis_api`。然后，点击 **Create API**。

我们已经创建了一个 API，但是它目前不能执行任何操作。我们希望它能触发之前创建的 Lambda 函数。

选择 **Actions** 下拉菜单并点击 **Create Method**。新的空方法将创建，打开下拉菜单并选择 **POST**，然后选中它旁边的复选框。

对于交互选项来说，选择 **Lambda Function** 并点击 **Use Lambda Proxy integration**。这个选项会让发送给 API 的数据直接发送给 Lambda 函数，不做任何处理。它还需要返回值必须是正确的响应对象，因为 API Gateway 也不会进行处理。

在 **Lambda Function** 文本框中输入之前创建的 Lambda 函数的名称，然后点击 **Save**。在出现的弹出式方框中点击 **OK**，使 API Gateway 有权调用你创建的 Lambda 函数。

创建 API Gateway 的最后一步是打开 **Actions** 下拉菜单并点击 **Deploy API**。你需要创建新的部署阶段并随意命名，例如 `prod`。

现在已经成功地设置了能访问 SageMaker 模型的公共 API。记得复制或写下调用新创建的公共 API 所需的 URL，因为在下一步将用到该 URL。你可以在页面顶部找到该 URL，它位于文字 **Invoke URL** 旁边，用蓝色标出。

## 第 4 步：部署网络应用

创建了可以公开访问的 API 后，我们可以在网络应用中使用它了。我们提供了简单的静态 HTML 文件，该文件可以使用你刚刚创建的公共 API。

在 `website` 文件夹里有一个文件 `index.html`。请将该文件下载到计算机上，并在任意文本编辑器中打开该文件。请将 **\*\*REPLACE WITH PUBLIC API URL\*\*** 替换成在上一步写下的 URL，然后保存文件。

现在在本地计算机上打开 `index.html`，浏览器将充当本地网络服务器，你可以使用提供的网站与 SageMaker 模型互动。

你还可以将此 HTML 文件托管到任何地方，例如 github，或将静态网站托管到 Amazon 的 S3 上。然后就可以将链接分享给任何人，让他们也能尝试该网站。

> **重要事项**：为了使网络应用能与 SageMaker 端点通信，你必须部署和运行端点。这样的话，就要付费了。当你想要使用网络应用的时候，请运行端点，但是不需要使用的时候，要关闭端点，否则会产生巨额 AWS 费用。

**TODO：**请在提交的项目中包含编辑后的 `index.html` 文件。

网络应用能正常运行了，尝试下该应用，看看运行效果如何。

**问题：**请提供一个你输入到网络应用中的影评示例。预测的情感是什么？

**回答：**

示例：I hardly watch a Hollywood movie but this movie was recommended by many. And now I would like to recommend this movie.

预测结果：


### 删除端点

如果不再使用端点，一定要关闭端点。端点按照运行时长计费，如果忘记关闭，最终可能会产生巨额费用。

In [53]:
predictor.delete_endpoint()